<a href="https://colab.research.google.com/github/cyuancheng/Predicting_Poverty/blob/master/04_Satellite_Image_process.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Code to read csv file into Colaboratory:
import os
import os.path
import numpy as np
import time
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt

import PIL
import PIL.Image
import tensorflow as tf

In [2]:
from fastai.vision import *
from fastai.widgets import *

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [18]:
path = Path('drive/My Drive')

In [19]:
# add all image filenames into a list
images_name = []
idx_name = []
for i in range(63):
  dir_ = str(path)+'/google_images/'+ str(i) + '/'
  try:
    image_files = os.listdir(dir_)
    images_name.append(image_files)
    idx_name.append(i)
  except: pass

In [20]:
all_images_downloaded = []
for i in range(0, len(images_name)): 
  list_within = images_name[i]
  for item in list_within: 
    all_images_downloaded.append(item)
print('total number of images =',len(all_images_downloaded))

total number of images = 50025


In [21]:
len_images_name=[]
for i in range(len(images_name)):
  len_images_name.append(len(images_name[i]))
#  print(len(images_name)):

In [22]:
df_test = pd.DataFrame({'idx':idx_name, 'names':len_images_name})
df_test.shape

(57, 2)

In [23]:
df_test['names'].sum()

50025

In [ ]:
### Classify images into three groups
def move_to_group(lightness_small, lightness_big, class_id):
    new_directory = 'data/google_image_cnn/class_' + str(class_id) + '/'
    if not os.path.isdir(new_directory):
        os.makedirs(new_directory)
    for i in range(lightness_small, lightness_big):
        path = 'data/google_image/' + str(i) + '/'
        for f in os.listdir(path):
            copyfile(path + f, new_directory + f)

move_to_group(0, 1, 1)
move_to_group(1, 10, 2)
move_to_group(10, 64, 3)

,idx,names
0,0,48960
1,6,332
2,7,240
3,8,122
4,9,91


### Merge daytime image data with DHS

In [24]:
features_basic = np.loadtxt(str(path)+'/google_images/google_image_features_basic.csv')

In [25]:
# help file
def get_cell_idx(lon, lat, top_left_x_coords, top_left_y_coords):
    lon_idx = np.where(top_left_x_coords < lon)[0][-1]
    lat_idx = np.where(top_left_y_coords > lat)[0][-1]
    return lon_idx, lat_idx

def get_daytime_feature(sample):
    idx, wealth, x, y = sample
    lon_idx, lat_idx = get_cell_idx(x, y, top_left_x_coords, top_left_y_coords)
    left_idx = lon_idx - 5
    right_idx = lon_idx + 4
    up_idx = lat_idx - 5
    low_idx = lat_idx + 4
    features_100 = []
    for i in range(left_idx, right_idx + 1):
        for j in range(up_idx, low_idx + 1):
            feature = features_basic[((features_basic[:,0]==i) & (features_basic[:,1]==j)),][:,2:]
            if len(feature) > 0:
                features_100.append(feature)
    if len(features_100) == 0:
        return np.asarray([np.nan] * 15 + [wealth]).tolist()
    else:
        features_all = np.concatenate(features_100, axis=0)
        mean_ = np.mean(features_all, axis=0).tolist()
        mean_.append(wealth)
        return mean_

In [26]:
npzfile = np.load(str(path)+'/google_images/nightlight.npz')
top_left_x_coords = npzfile['top_left_x_coords']
top_left_y_coords = npzfile['top_left_y_coords']
bands_data = npzfile['bands_data']

In [27]:
clusters = pd.read_csv(str(path)+'/google_images/BU_cluster_avg_asset_2010.csv')
clusters['feature'] = clusters.apply(lambda x: get_daytime_feature([x['cluster'], x['wlthindf'], x['longitude'], x['latitude']]), axis=1)

data_all = clusters['feature']
data_all = np.asarray([i for i in data_all])
data_all = data_all[~np.isnan(data_all).any(axis=1)]

np.savetxt(str(path)+'/google_images/DHS_daytime.csv', data_all)

In [29]:
#data_all = np.loadtxt('intermediate_files/DHS_daytime.csv')


Image Preprocess

In [31]:
image_path = 'drive/My Drive/google_image_class/'
def plt_display(image, title):
  fig = plt.figure()
  a = fig.add_subplot(1, 1, 1)
  imgplot = plt.imshow(image)
  a.set_title(title)

In [33]:
path_1 = Path('drive/My Drive/google_image_class/class_1')
np.random.seed(42)
data_1 = ImageDataBunch.from_folder(path_1, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)


In [ ]:
data_1.show_batch(rows=3, figsize=(8,8))

In [ ]:
path_2 = Path('drive/My Drive/google_image_class/class_2')
np.random.seed(42)
data_2 = ImageDataBunch.from_folder(path_2, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)


In [ ]:
data_2.show_batch(rows=3, figsize=(8,8))

In [ ]:
path_3 = Path('drive/My Drive/google_image_class/class_3')
np.random.seed(42)
data_3 = ImageDataBunch.from_folder(path_3, train=".", valid_pct=0.2,
        ds_tfms=get_transforms(), size=224, num_workers=4).normalize(imagenet_stats)


In [ ]:
data_3.show_batch(rows=3, figsize=(8,8))

Deep learning

Balance dataset

In [ ]:
# # Creating Train / Test folders 
root_dir = 'drive/My Drive/google_image_class/'
classes_dir = ['class_1', 'class_2', 'class_3']

for cls in classes_dir:
    os.makedirs(root_dir +cls+'_bal')
    src = root_dir + cls # Folder to copy images from
    allFileNames = os.listdir(src)
    np.random.choice(allFileNames, 238, replace=False) # balance the images

Create train test setss

In [ ]:
import os
import numpy as np
import shutil
import random

# # Creating Train / Val / Test folders (One time use)
root_dir = 'drive/My Drive/google_image_class/'
classes_dir = ['class_1', 'class_2', 'class_3']

test_ratio = 0.2

for cls in classes_dir:
    os.makedirs(root_dir +'train/' + cls)
    os.makedirs(root_dir +'test/' + cls)

    # Creating partitions of the data after shuffling
    src = root_dir + cls # Folder to copy images from

    allFileNames1 = os.listdir(src)
    allFileNames=np.random.choice(allFileNames1, 238, replace=False)
    #np.random.shuffle(allFileNames)
    train_FileNames, test_FileNames = np.split(np.array(allFileNames),[int(len(allFileNames)* (1 - test_ratio))])

    train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
    test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

    print('Total images: ', len(allFileNames))
    print('Training: ', len(train_FileNames))
    print('Testing: ', len(test_FileNames))

    # Copy-pasting images
    for name in train_FileNames:
        shutil.copy(name, root_dir +'train/' + cls)

    for name in test_FileNames:
        shutil.copy(name, root_dir +'test/' + cls)

Total images:  238
Training:  190
Testing:  48
Total images:  238
Training:  190
Testing:  48
Total images:  238
Training:  190
Testing:  48


Transfer learning (feature extraction)

In [ ]:
import numpy as np
from keras.preprocessing import image
from keras.models import Sequential
from tensorflow.keras.applications.imagenet_utils import preprocess_input, decode_predictions
from keras.layers.convolutional import Convolution2D, AveragePooling2D
from keras.optimizers import SGD
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Dropout
from keras.models import Model
from multiprocessing import Pool
import os
import time

In [ ]:
# load pre-trained model
from tensorflow.keras.applications.vgg16 import VGG16
model_vgg16 = VGG16(weights='imagenet', include_top=False)

58892288/58889256 [==============================] - 2s 0us/step


In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
model_ResNet50 = ResNet50(weights='imagenet')

102973440/102967424 [==============================] - 3s 0us/step


In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
model_inceptionV3 = InceptionV3(weights='imagenet', include_top=False)

87916544/87910968 [==============================] - 2s 0us/step


In [ ]:
def get_input_feature(img_path):
    # img = image.load_img(img_path, target_size=(400, 400))
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    features = model_inceptionV3.predict(x)
    return features[0]

In [ ]:
# train
all_figures = []
trainLabels = []

path_1 = 'drive/My Drive/google_image_class/train/class_1/'
class_1_files = os.listdir(path_1)
trainLabels += [[1, 0, 0]] * len(class_1_files)
all_figures += [path_1 + i for i in class_1_files]

path_2 = 'drive/My Drive/google_image_class/train/class_2/'
class_2_files = os.listdir(path_2)
trainLabels += [[0, 1, 0]] * len(class_2_files)
all_figures += [path_2 + i for i in class_2_files]

path_3 = 'drive/My Drive/google_image_class/train/class_3/'
class_3_files = os.listdir(path_3)
trainLabels += [[0, 0, 1]] * len(class_3_files)
all_figures += [path_3 + i for i in class_3_files]

In [ ]:
trainData = []
t1 = time.time()
for idx, i in enumerate(all_figures):
    a = get_input_feature(i)
    trainData.append(a)
    if idx % 100 == 0:
        t2 = time.time()
        print(idx)
        print((t2 - t1), 'sec')
        t1 = time.time()

0
0.03638488054275513 min
100
0.11752695242563883 min
200
0.12909011443456014 min
300
0.1283322016398112 min
400
0.12995003461837767 min
500
0.129706076780955 min


In [ ]:
x_all = np.asarray(trainData)
y_all = np.asarray(trainLabels)

# test
all_figures = []
testLabels = []

path_1 = 'drive/My Drive/google_image_class/test/class_1/'
# path_1 = 'google_image_cnn/class_1/'
class_1_files = os.listdir(path_1)
testLabels += [[1, 0, 0]] * len(class_1_files)
all_figures += [path_1 + i for i in class_1_files]

path_2 = 'drive/My Drive/google_image_class/test/class_2/'
# path_2 = 'google_image_cnn/class_2/'
class_2_files = os.listdir(path_2)
testLabels += [[0, 1, 0]] * len(class_2_files)
all_figures += [path_2 + i for i in class_2_files]

path_3 = 'drive/My Drive/google_image_class/test/class_3/'
# path_3 = 'google_image_cnn/class_3/'
class_3_files = os.listdir(path_3)
testLabels += [[0, 0, 1]] * len(class_3_files)
all_figures += [path_3 + i for i in class_3_files]

In [ ]:
testData = []
t1 = time.time()
for idx, i in enumerate(all_figures):
    a = get_input_feature(i)
    testData.append(a)
    if idx % 1000 == 0:
        t2 = time.time()
        print(idx)
        print(t2 - t1, 'sec')
        t1 = time.time()

x_all_test = np.asarray(testData)
y_all_test = np.asarray(testLabels)

0
0.08489322662353516


In [ ]:

# np.savez('google_image_feature.npz', x_all=x_all, y_all=y_all)
np.savez('google_image_feature_InceptionV3.npz', x_all=x_all, y_all=y_all, x_all_test=x_all_test, y_all_test=y_all_test)
